In [1]:
import numpy as np
import pandas as pd
from scipy.ndimage import rotate
from skimage.util import montage
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 
from keras.callbacks import CSVLogger
from tensorflow.keras import optimizers
import os
import nibabel as nib
import cv2 
from tensorflow.keras import backend as K
import glob
import skimage.io as io
import skimage.color as color
import random as r
import math
from nilearn import plotting
import warnings
from sklearn.preprocessing import MinMaxScaler


In [2]:
data  = np.load('/kaggle/input/aug3dn/input_data/train/train/image_1.npy')
print(data.shape)
data  = np.load('/kaggle/input/aug3dn/input_data/train/mask/image_1.npy')
print(data.shape)

(128, 128, 128, 3)
(128, 128, 128, 4)


In [3]:
def load_img(img_dir, img_list):
    images=[]
    for i, image_name in enumerate(img_list):    
        if (image_name.split('.')[1] == 'npy'):
            
            image = np.load(img_dir+image_name)         
            images.append(image)
    images = np.array(images)
    
    return(images)



def data_generator(img_dir, img_list, mask_dir, mask_list, batch_size):

    L = len(img_list)

    #keras needs the generator infinite, so we will use while true  
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
                       
               
            X = load_img(img_dir, img_list[batch_start:limit])
            Y = load_img(mask_dir, mask_list[batch_start:limit])
 
            
            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

############################################

In [4]:
train_img_dir = "/kaggle/input/aug3dn/input_data/train/train/"
train_mask_dir = "/kaggle/input/aug3dn/input_data/train/mask/"

val_img_dir = "/kaggle/input/aug3dn/input_data/val/train/"
val_mask_dir = "/kaggle/input/aug3dn/input_data/val/mask/"

train_img_list = os.listdir(train_img_dir)
train_mask_list = os.listdir(train_mask_dir)

val_img_list = os.listdir(val_img_dir)
val_mask_list = os.listdir(val_mask_dir)

number_img = len(os.listdir(train_img_dir))
               

In [5]:
batch_size = 1
train_img_gen = data_generator(train_img_dir, train_img_list, 
                                train_mask_dir, train_mask_list, batch_size)

val_img_gen = data_generator(val_img_dir, val_img_list, 
                                val_mask_dir, val_mask_list, batch_size)

In [6]:
# Check shape of a single batch from generators
batch_train = next(iter(train_img_gen))
print("Train batch shape:", batch_train[0].shape)

batch_val = next(iter(val_img_gen))
print("Validation batch shape:", batch_val[0].shape)


Train batch shape: (1, 128, 128, 128, 3)
Validation batch shape: (1, 128, 128, 128, 3)


In [7]:
import tensorflow as tf

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Calculate the Dice coefficient for binary segmentation.

    Parameters:
    y_true (tensor): Ground truth mask, shape (batch_size, depth, height, width, 1)
    y_pred (tensor): Predicted mask, shape (batch_size, depth, height, width, 1)
    smooth (float): Smoothing constant to prevent division by zero.

    Returns:
    tensor: Dice coefficient value.
    """
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    
    dice_score = (2.0 * intersection + smooth) / (union + smooth)
    return dice_score

def iou(y_true, y_pred, smooth=1e-6):
    """
    Calculate Intersection over Union (IoU) for binary segmentation.

    Parameters:
    y_true (tensor): Ground truth mask, shape (batch_size, depth, height, width, 1)
    y_pred (tensor): Predicted mask, shape (batch_size, depth, height, width, 1)
    smooth (float): Smoothing constant to prevent division by zero.

    Returns:
    tensor: IoU value.
    """
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    iou_score = (intersection + smooth) / (union + smooth)
    return iou_score

def precision(y_true, y_pred, smooth=1e-6):
    """
    Calculate precision for binary segmentation.

    Parameters:
    y_true (tensor): Ground truth mask, shape (batch_size, depth, height, width, 1)
    y_pred (tensor): Predicted mask, shape (batch_size, depth, height, width, 1)
    smooth (float): Smoothing constant to prevent division by zero.

    Returns:
    tensor: Precision value.
    """
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    true_positive = tf.reduce_sum(y_true * y_pred)
    predicted_positive = tf.reduce_sum(y_pred)

    precision_score = (true_positive + smooth) / (predicted_positive + smooth)
    return precision_score



In [8]:
pip install segmentation_models_3D

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
# !pip install segmentation-models-3D
wt0, wt1, wt2, wt3 = 0.25,0.25,0.25,0.25
import segmentation_models_3D as sm
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)



Segmentation Models: using `keras` framework.


In [10]:
steps_per_epoch = len(train_img_list)//batch_size
val_steps_per_epoch = len(val_img_list)//batch_size

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

def dense_block(x, num_filters):
    """Creates a dense block with DenseNet layers."""
    # Convolutional Layer
    conv = layers.Conv3D(num_filters, kernel_size=(3, 3, 3), padding='same')(x)
    conv = layers.BatchNormalization()(conv)
    conv = layers.ReLU()(conv)
    # Concatenate input and output of this layer
    return layers.concatenate([x, conv])

def transition_layer(x, num_filters):
    """Creates a transition layer with convolution and pooling."""
    x = layers.Conv3D(num_filters, kernel_size=(1, 1, 1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

def create_densenet_3d_unet(input_shape, num_classes):
    """Create a 3D U-Net model with DenseNet blocks."""
    inputs = layers.Input(shape=input_shape)

    # Encoder
    d1 = dense_block(inputs, 32)
    t1 = transition_layer(d1, 32)

    d2 = dense_block(t1, 64)
    t2 = transition_layer(d2, 64)

    d3 = dense_block(t2, 128)
    t3 = transition_layer(d3, 128)

    d4 = dense_block(t3, 256)
    t4 = transition_layer(d4, 256)

    # Bottleneck
    bottleneck = dense_block(t4, 512)

    # Decoder
    u4 = layers.Conv3DTranspose(256, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(bottleneck)
    u4 = layers.concatenate([u4, d4])
    c5 = dense_block(u4, 256)

    u3 = layers.Conv3DTranspose(128, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(c5)
    u3 = layers.concatenate([u3, d3])
    c6 = dense_block(u3, 128)

    u2 = layers.Conv3DTranspose(64, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(c6)
    u2 = layers.concatenate([u2, d2])
    c7 = dense_block(u2, 64)

    u1 = layers.Conv3DTranspose(32, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(c7)
    u1 = layers.concatenate([u1, d1])
    c8 = dense_block(u1, 32)

    outputs = layers.Conv3D(num_classes, kernel_size=(1, 1, 1), activation='softmax')(c8)

    model1 = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model1

# Example usage
input_shape = (128, 128, 128, 3)  # Change as needed (e.g., (depth, height, width, channels))
num_classes = 4  # Change to the number of classes in your dataset
model = create_densenet_3d_unet(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',dice_coefficient , precision ,iou])

In [12]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 128, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d (Conv3D)     │ (None, 128, 128,  │      2,624 │ input_layer[0][0] │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        128 │ conv3d[0][0]      │
│ (BatchNormalizatio… │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ input_layer[0][0… │
│ (Concatenate)       │ 128, 35)          │            │ re_lu[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_1 (Conv3D)   │ (None, 128, 128,  │      1,152 │ concatenate[0][0] │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        128 │ conv3d_1[0][0]    │
│ (BatchNormalizatio… │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 128, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d       │ (None, 64, 64,    │          0 │ re_lu_1[0][0]     │
│ (MaxPooling3D)      │ 64, 32)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 64, 64,    │     55,360 │ max_pooling3d[0]… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 64, 64,    │          0 │ max_pooling3d[0]… │
│ (Concatenate)       │ 64, 96)           │            │ re_lu_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 64, 64,    │      6,208 │ concatenate_1[0]… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 64, 64)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling3d_1     │ (None, 32, 32,    │          0 │ re_lu_3[0][0]   

 Total params: 13,484,592 (51.44 MB)

 Trainable params: 13,480,688 (51.42 MB)

 Non-trainable params: 3,904 (15.25 KB)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback to save the model with the best val_accuracy
checkpoint_callback = ModelCheckpoint(
    filepath='/kaggle/working/Den3dunetAttention.keras',  
    monitor='val_dice_coefficient',    
    save_best_only=True,       
    mode='max',                
    verbose=1                 
)

In [14]:
history1=model.fit(train_img_gen,
          steps_per_epoch=steps_per_epoch,
          epochs=20,
          verbose=1,
          validation_data=val_img_gen,
          validation_steps=val_steps_per_epoch,
          callbacks=[checkpoint_callback]
          )



Epoch 1/20


I0000 00:00:1745761265.226844      80 service.cc:145] XLA service 0x7919f4001e20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745761265.226914      80 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-04-27 13:41:17.553901: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng20{k2=6,k3=0} for conv (f32[32,3,3,3,3]{4,3,2,1,0}, u8[0]{0}) custom-call(f32[1,3,128,128,128]{4,3,2,1,0}, f32[1,32,128,128,128]{4,3,2,1,0}), window={size=3x3x3 pad=1_1x1_1x1_1}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2025-04-27 13:41:17.588607: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.034914183s
T

335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 690ms/step - accuracy: 0.9370 - dice_coefficient: 0.9145 - iou: 0.8706 - loss: 0.1774 - precision: 0.9145
Epoch 1: val_dice_coefficient improved from -inf to 0.95592, saving model to /kaggle/working/Den3dunetAttention.keras
335/335 ━━━━━━━━━━━━━━━━━━━━ 356s 924ms/step - accuracy: 0.9371 - dice_coefficient: 0.9146 - iou: 0.8707 - loss: 0.1771 - precision: 0.9146 - val_accuracy: 0.9654 - val_dice_coefficient: 0.9559 - val_iou: 0.9183 - val_loss: 0.1219 - val_precision: 0.9559
Epoch 2/20
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 681ms/step - accuracy: 0.9793 - dice_coefficient: 0.9700 - iou: 0.9422 - loss: 0.0590 - precision: 0.9700
Epoch 2: val_dice_coefficient did not improve from 0.95592
335/335 ━━━━━━━━━━━━━━━━━━━━ 304s 907ms/step - accuracy: 0.9793 - dice_coefficient: 0.9700 - iou: 0.9422 - loss: 0.0590 - precision: 0.9700 - val_accuracy: 0.8665 - val_dice_coefficient: 0.8533 - val_iou: 0.7722 - val_loss: 0.4177 - val_precision: 0.8533
Epoch 3/20
335/335 ━━━━━━━

In [15]:
model.save("net.keras")

In [18]:
from IPython.display import FileLink
import os

# Change directory to where your files are saved
os.chdir('/kaggle/working')

file_path_1 = 'Den3dunetAttention.keras'
file_path_2 = 'net.keras'

print("Generating download links. Please commit the notebook after running this cell.")

if os.path.exists(file_path_1):
    display(FileLink(file_path_1))
else:
    print(f"File not found: {file_path_1}")

if os.path.exists(file_path_2):
    display(FileLink(file_path_2))
else:
    print(f"File not found: {file_path_2}")

print("\nAfter committing the notebook, find these links in the 'Output' tab to download your models.")

Generating download links. Please commit the notebook after running this cell.


/kaggle/working/Den3dunetAttention.keras

/kaggle/working/net.keras


After committing the notebook, find these links in the 'Output' tab to download your models.
